# IEMS 450-2 Programming Assignment 2

Created: March 8, 2024

Author: Wenhao Gu

This is an implementation of the active set quadratic programming algorithm.

In [113]:
# Make NumPy module available for numerical computations
import numpy as np

In [114]:
class ActiveSetQP:

    '''
    Solve Inequality Constrained Quadratic Programming (QP) using Active Set Method.

    min  1/2*x^T*G*x + c^T*x
    s.t. Ax >= b

    A breif introducion to the object methods:
        __init__():
            Intialize problem data and algorithm parameters.

        get_search_direction():
            Get the direction of the next iteration.

        get_step_size_and_binding():
            Get the stepsize of the next iteration.

        solve():
            The main function to solve the QP.
            
    For details, see each method function.
    '''

    def __init__(self, G, c, A, b, x_0, W_0, max_iter=1e4, tol=1e-6, verbose=2):

        '''
        Get data for the QP and the algorithm options.

        Inputs:
            G: 
                The matrix G in the objection function of the QP.
            c: 
                The vector c in the objection function of the QP.
            A: 
                The matrix A in the constraints of the QP.
            b: 
                The vector b in the constraints of the QP.
            x_0:
                The starting point.
            W_0:
                Initial working set.
            max_iter:
                Maximum number of iterations.
            Tol:
                Tolerance for optimality.
            Verbose:
                Amount of output printed
                0: No output
                1: Only summary information
                2: One line per iteration
        '''
        
        self.G = G
        self.c = c
        self.A = A
        self.m = A.shape[0]
        self.n = A.shape[1]
        self.b = b
        self.x_0 = x_0
        self.W_0 = W_0
        self.max_iter = max_iter
        self.tol = tol
        self.verbose = verbose

    def get_search_direction(self, x_k, W_k):

        '''
        Compute the search direction p_k and the corresponding dual solution lmd_k.

        Inputs
            x_k: 
                Current solution.
            W_k: 
                Current working set.
        
        Return:
            p_k:
                The search direction.
            lmd_k:
                The dual solution to the EQP

        '''

        G = self.G
        c = self.c
        n = self.n
        m_W = round(sum(W_k))
        # Directly solve the KKT condition
        # construct big matrix M
        A_W = self.A[W_k>0.5,:]
        M = np.vstack((np.hstack((G,-A_W.T)), np.hstack((A_W, np.zeros((m_W,m_W))))))
        # construct rhs
        g_k = G @ x_k + c
        rhs = np.hstack([-g_k, np.zeros(m_W)])
        # get p_k and lmd_k
        sol = np.linalg.solve(M, rhs)
        p_k = sol[0:n]
        lmd_k = sol[n:n+m_W]
        return p_k, lmd_k
    
    def get_step_size_and_binding(self, x_k, p_k, W_k):

        '''
        Compute the stepsize alpha_k and the new binding constraint.

        Inputs
            x_k: 
                Current solution.
            p_k:
                The search direction.
            W_k: 
                Current working set.
        
        Return:
            alpha_k:
                Stepsize.
            idx:
                The index of the new binding constraints. 'None' if the are no new binding constraint.

        '''

        A = self.A
        b = self.b
        m = self.m
        arr1 = b - A @ x_k
        arr2 = A @ p_k
        arr3 = np.array([arr1[i]/arr2[i] if ((arr2[i]<0) and (W_k[i]==0)) else 2 for i in range(m)])
        alpha_k = min(1, min(arr3))
        if (min(arr3) < 1):
            idx = np.argmin(arr3)
        else:
            idx = None
        return alpha_k, idx
    
    def solve(self):

        '''
        Main method to solve the QP.
        x_sol, lmd_sol, W_sol, status, stats
        Return:
            x_sol:
                Optimal primal solution.
            lmd_sol:
                Optimal dual solution.
            W_sol:
                Optimal working set.
            status:
                Return code indicating reason for termination:
                0:  Optimal solution found
                -1:  Maximum number of iterations exceeded
                -99:  Unknown error
            stats:
                Dictionary with statistics for the run.  Its fields are
                norm_p         Norm of p_k
                num_iter       Number of iterations taken
                |W|            Size of the optimal working set
                obj            Optimal objective value
                p_feasi        Violation of primal feasibility
                d_feasi        Violation of dual feasibility
                stats['num_iter'] = k

        '''

        max_iter = self.max_iter
        status = -99
        x_k = self.x_0
        W_k = self.W_0
        k = 0
        alpha_k = 0
        W_change = 0

        # initial output
        if self.verbose >= 2:
            output_header = '%6s %23s %9s %9s %6s %9s' % \
                ('iter', 'f', '||p_k||', 'alpha', '#W_change', '|W_k|')

        ###########################
        # Beginning of main Loop
        ###########################

        # repeat util one of the termination conditions are satisfied
        while True:
            
            ######################################################
            # Compute search direction And Dual Variables
            ######################################################

            p_k, lmd_k = self.get_search_direction(x_k, W_k)
            p_k_norm = np.linalg.norm(p_k, np.inf)

            ##############################################################
            # Update Informations in This Iteration
            ##############################################################

            # iteration output
            if self.verbose >= 2:
                # Print the output header every 10 iterations
                if k % 10 == 0:
                    print(output_header)
                f_k = 0.5*np.inner(x_k, self.G @ x_k) + np.inner(self.c, x_k)
                print('%6i %23.16e %9.2e %9.2e %6d %9d' %
                    (k, f_k, p_k_norm, alpha_k, W_change, round(sum(W_k))))
                

            ######################################################
            # Check termination tests
            ######################################################

            # check optimality condition
            if ((p_k_norm < self.tol) and (min(lmd_k) > -self.tol)):
                # x_sol = x_k
                # W_sol = W_k
                # p_k_norm_sol = p_k_norm
                status = 0
                break
            
            # check if iteration exceeds maxiter
            if (k+1 > max_iter):
                status = -1
                break
            
            #####################################################
            # Update x_k and W_k and k
            ######################################################

            # update x_k and W_k
            if (p_k_norm < self.tol):
                # x_k is optimal to EQP, but some dual solution is negative.
                # Remove the working constraint with most negative dual solution and the smallest index.
                idx = np.where(W_k>0.5)[0][np.argmin(lmd_k)]
                W_k[idx] = 0
                # output of the change of W_k
                W_change = -(idx+1)
            else:
                # x_k is not optimal to EQP. Need to compute the direction to the optimal search direction
                # Find the biggest stepsize tp remain feasible
                alpha_k, idx = self.get_step_size_and_binding(x_k, p_k, W_k)
                x_k = x_k + alpha_k*p_k
                # if some constraints outside W_k becomes binding, add the one with the smallest index.
                if idx is not None:
                    W_k[idx] = 1
                    # output of the change of W_k
                    W_change = idx+1
                else:
                    # output of the change of W_k
                    W_change = 0

            # update iteration count
            k += 1

        ###########################
        # Finalize results
        ###########################

        # Set last iterate as the one that is returned, together with its objective
        # value
        x_sol = x_k
        W_sol = W_k
        f_sol = 0.5*np.inner(x_sol, self.G @ x_sol) + np.inner(self.c, x_sol)
        lmd_sol = np.zeros(self.m)
        lmd_sol[W_sol>0] = lmd_k
        p_feasi = max(0, max(self.b - self.A @ x_sol))
        d_feasi = np.linalg.norm(self.G @ x_sol + self.c - self.A.T @ lmd_sol, np.inf)

        # Set the statistics
        stats = {}
        stats['num_iter'] = k
        stats['|W|'] = round(sum(W_sol))
        stats['obj'] = f_sol
        stats['p_feasi'] = p_feasi
        stats['d_feasi'] = d_feasi

        # Final output message
        if self.verbose >= 1:
            print('')
            print('Number of iterations..............: %d' % k)
            print('Size of the working set...........: %d' % round(sum(W_sol)))
            print('Final objective...................: %g' % f_sol)
            print('Violation of primal feasibility...: %s' % p_feasi)
            print('Violation of dual feasibility.....: %s' % d_feasi)
            # print('||p_k|| at final point..........: %g' % p_k_norm)
            print('')
            if status == 0:
                print('Exit: Optimal solution found.')
            elif status == -1:
                print('Exit: Maximum number of iterations (%d) exceeded.' %
                    k)
            else:
                print('ERROR: Unknown status value: %d\n' % status)

        # Return output arguments
        return x_sol, lmd_sol, W_sol, status, stats

#### Instance (A)

In [115]:
G = np.array([[2,0],[0,2]])
c = np.array([-2, -5])
A = np.array([[1,-2], [-1,-2], [-1,2], [1, 0], [0, 1]])
b = np.array([-2, -6, -2, 0, 0])
x_0 = np.array([2,0])
W_0 = np.array([0, 0, 0, 0, 0])
qp = ActiveSetQP(G, c, A, b, x_0, W_0, verbose=2)
x_sol, lmd_sol, W_sol, status, stats = qp.solve()

  iter                       f   ||p_k||     alpha #W_change     |W_k|
     0  0.0000000000000000e+00  2.50e+00  0.00e+00      0         0
     1 -6.4444444444444446e+00  6.67e-02  6.67e-01      1         1
     2 -6.4499999999999993e+00  1.11e-16  1.00e+00      0         1

Number of iterations..............: 2
Size of the working set...........: 1
Final objective...................: -6.45
Violation of primal feasibility...: 0
Violation of dual feasibility.....: 2.220446049250313e-16

Exit: Optimal solution found.


The algorithm solves it very fast, using only 2 iterations.

#### Instance (B)

In [116]:
G = np.array([[2,0],[0,2]])
c = np.array([-2, -5])
A = np.array([[1,-2], [-1,-2], [-1,2], [1, 0], [0, 1]])
b = np.array([-2, -6, -2, 0, 0])
x_0 = np.array([2,0])
W_0 = np.array([0, 0, 1, 0, 1])
qp = ActiveSetQP(G, c, A, b, x_0, W_0, verbose=2)
x_sol, lmd_sol, W_sol, status, stats = qp.solve()

  iter                       f   ||p_k||     alpha #W_change     |W_k|
     0  0.0000000000000000e+00  6.66e-16  0.00e+00      0         2
     1  0.0000000000000000e+00  1.00e+00  0.00e+00     -3         1
     2 -1.0000000000000000e+00  0.00e+00  1.00e+00      0         1
     3 -1.0000000000000000e+00  2.50e+00  1.00e+00     -5         0
     4 -6.2500000000000000e+00  4.00e-01  6.00e-01      1         1
     5 -6.4500000000000011e+00  1.11e-16  1.00e+00      0         1

Number of iterations..............: 5
Size of the working set...........: 1
Final objective...................: -6.45
Violation of primal feasibility...: 0
Violation of dual feasibility.....: 2.220446049250313e-16

Exit: Optimal solution found.


Changing the initial working set result in need for more iterations.

#### Instance (C)

In [117]:
G = np.array([[2,0],[0,2]])
c = np.array([-2, -5])
A = np.array([[1,-2], [-1,-2], [-1,2], [1, 0], [0, 1]])
b = np.array([-2, -6, -2, 0, 0])
x_0 = np.array([2,0])
W_0 = np.array([0, 0, 1, 0, 0])
qp = ActiveSetQP(G, c, A, b, x_0, W_0, verbose=2)
x_sol, lmd_sol, W_sol, status, stats = qp.solve()

  iter                       f   ||p_k||     alpha #W_change     |W_k|
     0  0.0000000000000000e+00  2.00e-01  0.00e+00      0         1
     1 -4.9999999999999822e-02  2.22e-16  1.00e+00      0         1
     2 -4.9999999999999822e-02  2.40e+00  1.00e+00     -3         0
     3 -6.4500000000000011e+00  1.11e-16  6.67e-01      1         1

Number of iterations..............: 3
Size of the working set...........: 1
Final objective...................: -6.45
Violation of primal feasibility...: 0
Violation of dual feasibility.....: 2.220446049250313e-16

Exit: Optimal solution found.


Changing the initial working set also decreases the number of iterations needed.

#### Instance (D)

In [118]:
G = np.array([[2,0],[0,2]])
c = np.array([-2, -5])
A = np.array([[1,-2], [-1,-2], [-1,2], [1, 0], [0, 1]])
b = np.array([-2, -6, -2, 0, 0])
x_0 = np.array([2,0])
W_0 = np.array([0, 0, 0, 0, 1])
qp = ActiveSetQP(G, c, A, b, x_0, W_0, verbose=2)
x_sol, lmd_sol, W_sol, status, stats = qp.solve()

  iter                       f   ||p_k||     alpha #W_change     |W_k|
     0  0.0000000000000000e+00  1.00e+00  0.00e+00      0         1
     1 -1.0000000000000000e+00  0.00e+00  1.00e+00      0         1
     2 -1.0000000000000000e+00  2.50e+00  1.00e+00     -5         0
     3 -6.2500000000000000e+00  4.00e-01  6.00e-01      1         1
     4 -6.4500000000000011e+00  1.11e-16  1.00e+00      0         1

Number of iterations..............: 4
Size of the working set...........: 1
Final objective...................: -6.45
Violation of primal feasibility...: 0
Violation of dual feasibility.....: 2.220446049250313e-16

Exit: Optimal solution found.


Changing the initial working set may change the convergence rate.

#### Instance (E)

In [119]:
G = np.array([[2,0],[0,2]])
c = np.array([-6, -4])
A = np.array([[-1,-1], [1,0], [0,1]])
b = np.array([-3, 0, 0])
x_0 = np.array([0,0])
W_0 = np.array([0, 0, 0])
qp = ActiveSetQP(G, c, A, b, x_0, W_0, verbose=2)
x_sol, lmd_sol, W_sol, status, stats = qp.solve()

  iter                       f   ||p_k||     alpha #W_change     |W_k|
     0  0.0000000000000000e+00  3.00e+00  0.00e+00      0         0
     1 -1.0919999999999998e+01  2.00e-01  6.00e-01      1         1
     2 -1.1000000000000000e+01  0.00e+00  1.00e+00      0         1

Number of iterations..............: 2
Size of the working set...........: 1
Final objective...................: -11
Violation of primal feasibility...: 0
Violation of dual feasibility.....: 0.0

Exit: Optimal solution found.


Too little iterations

#### Instance (F)

In [120]:
G = np.array([[2,0],[0,2]])
c = np.array([-6, -4])
A = np.array([[-1,-1], [1,0], [0,1]])
b = np.array([-3, 0, 0])
x_0 = np.array([0,0])
W_0 = np.array([0, 1, 1])
qp = ActiveSetQP(G, c, A, b, x_0, W_0, verbose=2)
x_sol, lmd_sol, W_sol, status, stats = qp.solve()

  iter                       f   ||p_k||     alpha #W_change     |W_k|
     0  0.0000000000000000e+00  0.00e+00  0.00e+00      0         2
     1  0.0000000000000000e+00  3.00e+00  0.00e+00     -2         1
     2 -9.0000000000000000e+00  0.00e+00  1.00e+00      0         1
     3 -9.0000000000000000e+00  2.00e+00  1.00e+00     -3         0
     4 -9.0000000000000000e+00  1.00e+00 -0.00e+00      1         1
     5 -1.1000000000000000e+01  0.00e+00  1.00e+00      0         1

Number of iterations..............: 5
Size of the working set...........: 1
Final objective...................: -11
Violation of primal feasibility...: 0
Violation of dual feasibility.....: 0.0

Exit: Optimal solution found.


Too little iterations

#### Instance (G)

In [121]:
G = np.array([[2,0],[0,2]])
c = np.array([-6, -4])
A = np.array([[-1,-1], [1,0], [0,1]])
b = np.array([-3, 0, 0])
x_0 = np.array([1,1])
W_0 = np.array([0, 0, 0])
qp = ActiveSetQP(G, c, A, b, x_0, W_0, verbose=2)
x_sol, lmd_sol, W_sol, status, stats = qp.solve()

  iter                       f   ||p_k||     alpha #W_change     |W_k|
     0 -8.0000000000000000e+00  2.00e+00  0.00e+00      0         0
     1 -1.0777777777777777e+01  3.33e-01  3.33e-01      1         1
     2 -1.1000000000000000e+01  0.00e+00  1.00e+00      0         1

Number of iterations..............: 2
Size of the working set...........: 1
Final objective...................: -11
Violation of primal feasibility...: 0
Violation of dual feasibility.....: 0.0

Exit: Optimal solution found.


Too little iterations

#### Instance (H)

In [122]:
G = np.array([[6,2,1],[2,5,2],[1,2,4]])
c = np.array([-8, -3, -3])
A = np.array([[1,0,1], [-1,0,-1], [0,1,1], [0,-1,-1]])
b = np.array([2.999, -3.001, -0.001, -0.001])
x_0 = np.array([3,0,0])
W_0 = np.array([0, 0, 0, 0])
qp = ActiveSetQP(G, c, A, b, x_0, W_0, verbose=2)
x_sol, lmd_sol, W_sol, status, stats = qp.solve()

  iter                       f   ||p_k||     alpha #W_change     |W_k|
     0  3.0000000000000000e+00  1.71e+00  0.00e+00      0         0
     1  2.9858194973567826e+00  1.25e+00  8.14e-04      1         1
     2  2.9705264326923064e+00  9.98e-01  1.07e-03      4         2
     3 -3.5049966538461561e+00  1.39e-16  1.00e+00      0         2

Number of iterations..............: 3
Size of the working set...........: 2
Final objective...................: -3.505
Violation of primal feasibility...: 8.881784197001252e-16
Violation of dual feasibility.....: 4.440892098500626e-16

Exit: Optimal solution found.


Too little iterations

#### Instance (I)

In [123]:
np.random.seed(0)
n = 10
m = 10
L = np.random.rand(n,n)
G = np.matmul(L,L.T)
c = 10*(0.5-np.random.rand(n))
A = 10*(0.5-np.random.rand(m,n))
b = 10*(0.5-np.random.rand(m))
x_feas = np.random.rand(n)
b = A @ x_feas - 1
W_0 = np.zeros(m, dtype=np.int64)

qp = ActiveSetQP(G, c, A, b, x_feas, W_0, verbose=2)
x_sol, lmd_sol, W_sol, status, stats = qp.solve()

  iter                       f   ||p_k||     alpha #W_change     |W_k|
     0  4.1271903801539665e+01  2.65e+02  0.00e+00      0         0
     1  3.9973631363183500e+01  7.04e+01  9.92e-04      9         1
     2  3.9936895675736274e+01  2.23e+01  8.26e-05      7         2
     3  3.8858079612820383e+01  1.83e+01  4.23e-03      5         3
     4  3.4025463836612040e+01  1.46e+01  2.05e-02      2         4
     5  2.8624075947470175e+01  1.30e+01  2.48e-02     10         5
     6 -4.6007304752265064e+01  1.05e-13  1.00e+00      0         5

Number of iterations..............: 6
Size of the working set...........: 5
Final objective...................: -46.0073
Violation of primal feasibility...: 1.0658141036401503e-14
Violation of dual feasibility.....: 3.6415315207705135e-14

Exit: Optimal solution found.


Alpha became very small but recover to 1.

#### Instance (J)

In [124]:
np.random.seed(0)
n = 100
m = 10
L = np.random.rand(n,n)
G = np.matmul(L,L.T)
c = 10*(0.5-np.random.rand(n))
A = 10*(0.5-np.random.rand(m,n))
b = 10*(0.5-np.random.rand(m))
x_feas = np.random.rand(n)
b = A @ x_feas - 1
W_0 = np.zeros(m, dtype=np.int64)

qp = ActiveSetQP(G, c, A, b, x_feas, W_0, verbose=2)
x_sol, lmd_sol, W_sol, status, stats = qp.solve()

  iter                       f   ||p_k||     alpha #W_change     |W_k|
     0  3.1017700303106587e+04  6.79e+02  0.00e+00      0         0
     1  3.1014851061501879e+04  1.86e+02  4.23e-05      1         1
     2  3.0981916707084634e+04  1.42e+02  5.19e-04      6         2
     3  3.0971122905956017e+04  6.14e+01  1.71e-04      4         3
     4  3.0967341522080016e+04  2.65e+01  6.01e-05      2         4
     5  3.0556626760240095e+04  2.04e+01  6.59e-03      7         5
     6 -2.6207298847149013e+02  1.21e-11  1.00e+00      0         5

Number of iterations..............: 6
Size of the working set...........: 5
Final objective...................: -262.073
Violation of primal feasibility...: 1.3855583347321954e-13
Violation of dual feasibility.....: 2.4011903576592886e-12

Exit: Optimal solution found.


Increase n doesn't significantly increase the number of iterations.

#### Instance (k)

In [125]:
np.random.seed(0)
n = 10
m = 100
L = np.random.rand(n,n)
G = np.matmul(L,L.T)
c = 10*(0.5-np.random.rand(n))
A = 10*(0.5-np.random.rand(m,n))
b = 10*(0.5-np.random.rand(m))
x_feas = np.random.rand(n)
b = A @ x_feas - 1
W_0 = np.zeros(m, dtype=np.int64)

qp = ActiveSetQP(G, c, A, b, x_feas, W_0, verbose=2)
x_sol, lmd_sol, W_sol, status, stats = qp.solve()

  iter                       f   ||p_k||     alpha #W_change     |W_k|
     0  2.3324977269697605e+01  2.66e+02  0.00e+00      0         0
     1  2.2907348184947374e+01  1.14e+01  3.28e-04     42         1
     2  2.2669059837682763e+01  1.20e+01  1.63e-03     99         2
     3  2.2247503468517820e+01  8.55e+00  3.64e-03     94         3
     4  2.2190827098072596e+01  6.72e+00  5.94e-04     93         4
     5  2.2017506641309634e+01  5.64e+00  2.29e-03     16         5
     6  2.1500210150119983e+01  1.15e+00  6.97e-03     90         6
     7  2.0363602673590513e+01  1.22e+00  3.50e-02     60         7
     8  1.9466712603773477e+01  1.52e+00  3.27e-02     29         8
     9  1.8718834151376626e+01  1.18e+00  3.17e-02     22         9
  iter                       f   ||p_k||     alpha #W_change     |W_k|
    10  1.8657776685789649e+01  3.21e-16  8.86e-03     68        10
    11  1.8657776685789649e+01  1.51e+00  8.86e-03    -94         9
    12  1.8390803048502402e+01  4.49e-16  

Increase m significantly increase the number of iterations, because the possible combinations of working sets increased a lot.

#### Instance (L)

In [126]:
np.random.seed(0)
n = 100
m = 200
L = np.random.rand(n,n)
G = np.matmul(L,L.T)
c = 10*(0.5-np.random.rand(n))
A = 10*(0.5-np.random.rand(m,n))
b = 10*(0.5-np.random.rand(m))
x_feas = np.random.rand(n)
b = A @ x_feas - 1
W_0 = np.zeros(m, dtype=np.int64)

qp = ActiveSetQP(G, c, A, b, x_feas, W_0, verbose=2)
x_sol, lmd_sol, W_sol, status, stats = qp.solve()

  iter                       f   ||p_k||     alpha #W_change     |W_k|
     0  2.5093390935182128e+04  6.79e+02  0.00e+00      0         0
     1  2.5091109460317548e+04  2.34e+02  4.11e-05     79         1
     2  2.5090670478623506e+04  1.14e+02  8.38e-06      1         2
     3  2.5082831309318746e+04  8.56e+01  1.53e-04     32         3
     4  2.5081994005903300e+04  6.08e+01  1.63e-05    166         4
     5  2.5064202537850342e+04  1.08e+01  3.48e-04    197         5
     6  2.5023937509988606e+04  1.15e+01  7.96e-04     86         6
     7  2.5007539027466886e+04  9.83e+00  3.25e-04    124         7


     8  2.4974951213007178e+04  1.16e+01  6.46e-04    130         8
     9  2.4957544931490003e+04  1.19e+01  3.46e-04     23         9
  iter                       f   ||p_k||     alpha #W_change     |W_k|
    10  2.4931650609832333e+04  7.38e+00  5.15e-04    135        10
    11  2.4918939288420988e+04  5.76e+00  2.54e-04    179        11
    12  2.4884625325863104e+04  6.01e+00  6.86e-04     87        12
    13  2.4799899966474484e+04  5.12e+00  1.70e-03     41        13
    14  2.4777086067667031e+04  4.60e+00  4.58e-04     35        14
    15  2.4712359035513629e+04  3.88e+00  1.30e-03     63        15
    16  2.4657395471990010e+04  4.31e+00  1.11e-03     45        16
    17  2.4648547541929853e+04  4.33e+00  1.79e-04     48        17
    18  2.4604273066462159e+04  3.69e+00  8.95e-04    147        18
    19  2.4595901484687467e+04  3.62e+00  1.70e-04     89        19
  iter                       f   ||p_k||     alpha #W_change     |W_k|
    20  2.4573134181179117e+04  3.51e+00  

#### Instance (M)

In [127]:
np.random.seed(0)
n = 200
m = 100
L = np.random.rand(n,n)
G = np.matmul(L,L.T)
c = 10*(0.5-np.random.rand(n))
A = 10*(0.5-np.random.rand(m,n))
b = 10*(0.5-np.random.rand(m))
x_feas = np.random.rand(n)
b = A @ x_feas - 1
W_0 = np.zeros(m, dtype=np.int64)

qp = ActiveSetQP(G, c, A, b, x_feas, W_0, verbose=2)
x_sol, lmd_sol, W_sol, status, stats = qp.solve()

  iter                       f   ||p_k||     alpha #W_change     |W_k|
     0  2.2583218223540703e+05  5.94e+02  0.00e+00      0         0
     1  2.2580592526095879e+05  3.34e+02  5.75e-05     26         1
     2  2.2577902450070440e+05  8.77e+01  5.91e-05      4         2
     3  2.2576991134534284e+05  6.64e+01  2.01e-05     55         3
     4  2.2572322325693379e+05  4.95e+01  1.03e-04     28         4
     5  2.2568832024464474e+05  4.38e+01  7.70e-05     29         5
     6  2.2562301821309939e+05  3.66e+01  1.44e-04     94         6
     7  2.2553068848875942e+05  2.47e+01  2.04e-04     42         7
     8  2.2551585390901600e+05  1.28e+01  3.28e-05     56         8
     9  2.2542156005495653e+05  1.29e+01  2.09e-04     60         9
  iter                       f   ||p_k||     alpha #W_change     |W_k|
    10  2.2532028596999560e+05  1.69e+01  2.24e-04     16        10
    11  2.2525756914272401e+05  1.78e+01  1.39e-04     47        11
    12  2.2509668616115258e+05  1.84e+01  

m has higher impact on the number of iterations needed than n.

#### Instance (N)

In [128]:
np.random.seed(0)
n = 1000
m = 1000
L = np.random.rand(n,n)
G = np.matmul(L,L.T)
c = 10*(0.5-np.random.rand(n))
A = 10*(0.5-np.random.rand(m,n))
b = 10*(0.5-np.random.rand(m))
x_feas = np.random.rand(n)
b = A @ x_feas - 1
W_0 = np.zeros(m, dtype=np.int64)

qp = ActiveSetQP(G, c, A, b, x_feas, W_0, verbose=2)
x_sol, lmd_sol, W_sol, status, stats = qp.solve()

  iter                       f   ||p_k||     alpha #W_change     |W_k|
     0  3.1095236154337686e+07  6.74e+03  0.00e+00      0         0
     1  3.1095115037508246e+07  1.10e+03  1.94e-06    212         1
     2  3.1094943358831208e+07  8.25e+02  2.76e-06    356         2
     3  3.1094785942600846e+07  4.49e+02  2.53e-06    575         3
     4  3.1094613476859942e+07  3.53e+02  2.77e-06    269         4
     5  3.1094424858751919e+07  3.18e+02  3.03e-06    713         5
     6  3.1094329148537640e+07  3.96e+02  1.54e-06    749         6
     7  3.1094258625590302e+07  2.44e+02  1.13e-06    887         7
     8  3.1094049254518282e+07  2.00e+02  3.37e-06    691         8
     9  3.1093577690703005e+07  1.88e+02  7.58e-06    279         9
  iter                       f   ||p_k||     alpha #W_change     |W_k|
    10  3.1093413683073226e+07  1.75e+02  2.64e-06    431        10
    11  3.1092609875842609e+07  1.27e+02  1.29e-05    244        11
    12  3.1092403953818120e+07  6.98e+01  

Number of iterations significant increases when m and n are very large.